In [ ]:
import sys
sys.path.append('/Users/liangwang/GitHub/DataAnalysis')

In [ ]:
import numpy as np
import os
from pathlib import Path
import pathlib
import platform
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None
import copy
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import n_colors
import math
import datetime as dt
from sklearn.metrics import r2_score
import plotly.colors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from scipy.interpolate import griddata
from scipy import signal 
from scipy import odr 
from itertools import combinations
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from statsmodels.regression.rolling import RollingOLS

# ----- Internal Dependencies -------#
nb_dir = os.path.split(os.getcwd())[0]
sys.path.append("../..")
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from General import FileRead
from General import AccuracyMetrics
from General.GeneralFunctions import get_add_to_dict
from General import TableManipulations
from FeatureGeneration import GeneralFeatures
from General import FileWrite

from Plot.PdfHelper import PdfHelper
from Plot.PlotMaker import PlotMaker
# from Plot import Voltage_Plots
from Plot import PlotFunctions
# from Plot import AdHocPlots
# from Plot import Sim_Plots

# widget and notebook stuff
from ipywidgets import interact, fixed
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

In [ ]:
pathlib.WindowsPath = pathlib.PosixPath

In [ ]:
biolinq_col_keep = ['ms_since_boot','algo_state','algo_state_desc','rate_of_change','is_rate_of_change_valid',
                    'time_from_start','biolinq','biolinq_s1','biolinq_s3','biolinq_s4','biolinq_no_blank','sensor_id','UTC_Time',
                    'color_code','color_desc']
reference_setting = {
    'ysi': {
        'time_to_match': 'UTC_Time',
        'direction_to_match': 'forward',
        'tol_to_match': '5min',
        'value_lb': 0,
        'value_ub': 900
    },
    'acck': {
        'time_to_match': 'UTC_Time',
        'direction_to_match': 'forward',
        'tol_to_match': '5min',
        'value_lb': 10,
        'value_ub': 600
    },
    'Libre': {
        'time_to_match': 'UTC_Time',
        'direction_to_match': 'forward',
        'tol_to_match': '5min',
        'value_lb': 40,
        'value_ub': 400
    },
    'Dexcom': {
        'time_to_match': 'UTC_Time',
        'direction_to_match': 'forward',
        'tol_to_match': '5min',
        'value_lb': 40,
        'value_ub': 400
    }         
}

In [ ]:
algo_data_dict = {
    # 'model_by_ysi':{
    #     'data_root_path': '/Volumes/Shared/Algo/Algorithm Output/Isabella_v02 time varying trained by ysi',
    #     'study_folder': ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','Pre-Piv-2-training','iBlinqAcet','iBlinqFW231','iBlinqOverlay']       
    # },
    # 'model_by_acck':{
    #     'data_root_path': '/Volumes/Shared/Algo/Algorithm Output/Isabella_v02 time varying trained by acck',
    #     'study_folder': ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','Pre-Piv-2-training','iBlinqAcet','iBlinqFW231','iBlinqOverlay']
    # },
    # 'model_by_ysicgm':{
    #     'data_root_path': '/Volumes/Shared/Algo/Algorithm Output/Isabella_v02 time varying trained by ysi and cgm',
    #     'study_folder': ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','Pre-Piv-2-training','iBlinqAcet','iBlinqFW231','iBlinqOverlay']
    # },
    # 'model_by_ysiacck':{
    #     'data_root_path': '/Volumes/Shared/Algo/Algorithm Output/Isabella_v02 time varying trained by ysi and acck',
    #     'study_folder': ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','Pre-Piv-2-training','iBlinqAcet','iBlinqFW231','iBlinqOverlay']
    # },    
    '6b6d8c6':{
        'data_root_path': '/Volumes/Shared/Algo/Algorithm Output/Isabella_v02 time varying trained by ysi and cgm',
        'study_folder': ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','Pre-Piv-2-training','iBlinqAcet']
    },
    '835b8ea':{
        'data_root_path': '/Volumes/Shared/Algo/Algorithm Output/Isabella_v02 time varying trained by ysi and acck',
        'study_folder': ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','Pre-Piv-2-training','iBlinqAcet']
    },
    'd4f86a1':{
        'data_root_path': '/Volumes/Shared/Algo/Algorithm Output/Isabella_v02 time varying trained by ysi and cgm',
        'study_folder': ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','Pre-Piv-2-training','iBlinqAcet']
    },
    '37d91eb':{
        'data_root_path': '/Volumes/Shared/Algo/Algorithm Output/Isabella_v02 time varying trained by ysi and acck',
        'study_folder': ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','Pre-Piv-2-training','iBlinqAcet']
    },                
}
save_agg_biolinq_ref_df = True
save_agg_biolinq_datastore = True
folder_to_save = '/Users/liangwang/Library/CloudStorage/OneDrive-Biolinq/Gen 1/Algorithm Development/Gen 1 Model Comparison'
agg_biolinq_ref_df_file_name = 'agg_biolinq_ref_df_tree_size_compare.csv'
agg_biolinq_datastore_file_name = 'agg_biolinq_datastore_tree_size_compare.csv'
agg_biolinq_ref_df_file_path = os.path.join(folder_to_save,agg_biolinq_ref_df_file_name)
agg_biolinq_datastore_file_path = os.path.join(folder_to_save,agg_biolinq_datastore_file_name)

In [ ]:
head_agg_biolinq_ref_df = False
head_agg_biolinq_datastore = False
for algo,algo_info in algo_data_dict.items():
    print(f'Processing data by algo: {algo}')
    data_root_path = algo_info['data_root_path']
    study_folder = algo_info['study_folder']
    for study in study_folder:
        print(f'Processing data from clinical trial: {study}')
        study_full_path = os.path.join(data_root_path,study)
        loaded_data = FileRead.load_pickle('alg_out.zip', study_full_path, as_dict=True)
        all_sensor_data = loaded_data['all_sensor_data']
        for sensor in all_sensor_data.keys():
            print(f'Processing data from sensor: {sensor}')
            current_sensor_data = all_sensor_data[sensor].copy()
            biolinq_exist = False
            ref_exist = {}
            ref_exist['acck_exist'] = False
            ref_exist['ysi_exist'] = False
            ref_exist['Libre_exist'] = False
            ref_exist['Dexcom_exist'] = False
            ref_df_dict = {}
            biolinq_ref_df_dict = {}
            if 'biolinq' in current_sensor_data.keys():
                biolinq_exist = True
                biolinq_df = current_sensor_data['biolinq'][biolinq_col_keep].copy()
                biolinq_datastore = current_sensor_data['biolinq_datastore'].copy()
                if biolinq_df.empty:
                    biolinq_exist = False
            
            if not biolinq_exist:
                continue
            biolinq_df['Algo'] = algo
            biolinq_df['Study'] = study
            biolinq_datastore['Algo'] = algo
            biolinq_datastore['Study'] = study
            #Fill in desired columns with NaN
            for col in biolinq_col_keep:
                if col not in biolinq_df.columns:
                    biolinq_df[col]=np.nan
                            
            for this_ref_name,this_ref_setting in reference_setting.items():
                if this_ref_name in current_sensor_data.keys():
                    ref_exist[f'{this_ref_name}_exist'] = True
                    ref_df_dict[f'{this_ref_name}_df'] = current_sensor_data[this_ref_name].copy()
                    if ref_df_dict[f'{this_ref_name}_df'].empty:
                        ref_exist[f'{this_ref_name}_exist'] = False
                    else:
                        ref_df_dict[f'{this_ref_name}_df'].rename(columns={this_ref_name:'ref_value'},inplace=True)
                        ref_df_dict[f'{this_ref_name}_df']['ref_type'] = this_ref_name
                        valid_idx = ref_df_dict[f'{this_ref_name}_df']['ref_value'].ge(this_ref_setting['value_lb']) \
                            & ref_df_dict[f'{this_ref_name}_df']['ref_value'].le(this_ref_setting['value_ub'])
                        ref_df_dict[f'{this_ref_name}_df'] = ref_df_dict[f'{this_ref_name}_df'].loc[valid_idx,:]
                if not ref_exist[f'{this_ref_name}_exist']:
                    continue
                time_to_match = this_ref_setting['time_to_match']
                direction_to_match = this_ref_setting['direction_to_match']
                tol_to_match = this_ref_setting['tol_to_match']
                biolinq_ref_df_dict[f'biolinq_{this_ref_name}_df'] = pd.merge_asof(left=ref_df_dict[f'{this_ref_name}_df'].dropna(subset=[time_to_match]).sort_values(by=time_to_match), right=biolinq_df.dropna(subset=[time_to_match]).sort_values(by=time_to_match),
                                            on=time_to_match, direction=direction_to_match, tolerance=pd.Timedelta(tol_to_match))

                #Save matched pairs table into CSV per reference
                if save_agg_biolinq_ref_df:
                    if head_agg_biolinq_ref_df==False:
                        biolinq_ref_df_dict[f'biolinq_{this_ref_name}_df'].to_csv(agg_biolinq_ref_df_file_path,mode='a',float_format='%g',index=False,header=True)
                        head_agg_biolinq_ref_df=True
                    else:
                        biolinq_ref_df_dict[f'biolinq_{this_ref_name}_df'].to_csv(agg_biolinq_ref_df_file_path,mode='a',float_format='%g',index=False,header=False) 

            # Save datastore table into csv per sensor
            if save_agg_biolinq_datastore:
                if head_agg_biolinq_datastore==False:
                    biolinq_datastore.to_csv(agg_biolinq_datastore_file_path,mode='a',float_format='%g',index=False,header=True)
                    head_agg_biolinq_datastore=True
                else:
                    biolinq_datastore.to_csv(agg_biolinq_datastore_file_path,mode='a',float_format='%g',index=False,header=False)